In [ ]:
from roboflow import Roboflow
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
import os
import shutil
import pandas as pd


In [7]:
rf = Roboflow(api_key="9XPp8Mmd8A0ZbBRONev4")
project = rf.workspace("yolo-dataset-lx2qb").project("classification-aiubq")
version = project.version(1)
# dataset = version.download("multiclass")
dataset = project.version(1).download("folder")

loading Roboflow workspace...
loading Roboflow project...
Exporting format folder in progress : 85.0%
Version export complete for folder format


In [10]:
print("Current working dir:", os.getcwd())
print("Folders inside project:", os.listdir())

Current working dir: c:\Users\user\Desktop\NTI -CV\Tech\S14\model resnet trasfer
Folders inside project: ['Classification-1', 'cvproject', 'VehicleNet.ipynb']


In [11]:
print("Inside Classification-1:", os.listdir("Classification-1"))
print("Inside train:", os.listdir("Classification-1/train"))

Inside Classification-1: ['README.dataset.txt', 'README.roboflow.txt', 'test', 'train', 'valid']
Inside train: ['10_png.rf.117ed06005bf2490fe557d9b1eadb6da.jpg', '10_png.rf.14ef6ff19a6ce04503220e48009d293d.jpg', '10_png.rf.748a6e16686b829d760b428acf238393.jpg', '10_png.rf.74c58b5c314806df6f0bb9d03d4df5c6.jpg', '10_png.rf.832b5fa24367ba313706c24bdd24fdd2.jpg', '10_png.rf.8be3da3a68181ed9ebb2c7b97738b55c.jpg', '10_png.rf.8e80963e78f093edc46ad67bf1b8dfb6.jpg', '10_png.rf.c6b2e50ebbe052757032f70b8fdf82e3.jpg', '10_png.rf.e52274d4b3823aa24bed404c96f703ed.jpg', '11_jpg.rf.6ccb244c5e81a1c6acf1a23f3f6edeb3.jpg', '11_jpg.rf.e114c788308c99786cb79a0508acbf28.jpg', '11_jpg.rf.fa7c99377b8d81c2f82a0f082daf3663.jpg', '11_png.rf.955a90caabb5f5b0f1ff04f57de90d58.jpg', '11_png.rf.e686734fefbef0b24cde522ddfcd5127.jpg', '11_png.rf.f89452c00853b53716734b2d38031114.jpg', '12_png.rf.1452a400bfbd1b000f3f8ce98483a144.jpg', '12_png.rf.5a6656bae5f1ffd4716e0eb2d31fcb3f.jpg', '12_png.rf.895906180f2a141ed881a985789

In [14]:
import pandas as pd
classes_df = pd.read_csv("Classification-1/train/_classes.csv")
print(classes_df)

                                              filename  bus  car  motorcycle  \
0    download-2-_jpg.rf.58b6099947d04ee7fff90762648...    0    1           0   
1    Image_013475_jpg.rf.7efcd9d59df04a813b2c561a44...    0    0           0   
2    OIP-18-_jpg.rf.d7242d61bbe21fb622cef3fa12ae83a...    0    1           0   
3    OIP-5-_jpg.rf.f2c455fe18c5930bb0fde33213a223b3...    0    1           0   
4    OIP-16-_jpg.rf.f8936c192136b50eeff4e1b7045e22e...    0    1           0   
..                                                 ...  ...  ...         ...   
415  20140413204213-30908023-me_jpg.rf.c1c739eb1426...    1    0           0   
416     22_png.rf.6e5f37362e7b70c3f0bbed07fc561757.jpg    0    0           1   
417     10_png.rf.8e80963e78f093edc46ad67bf1b8dfb6.jpg    0    0           1   
418     19_png.rf.7ed56da9a958ff628c384065805738cb.jpg    0    0           1   
419     15_jpg.rf.c808e80c3e6d5f48d2116324dd40dbe3.jpg    0    0           1   

     truck  
0        0  
1        1  


In [21]:
def organize_dataset(split_path):
    csv_path = os.path.join(split_path, "_classes.csv")
    df = pd.read_csv(csv_path)

    class_columns = df.columns[1:]

    for _, row in df.iterrows():
        filename = row["filename"]

        label = None
        for col in class_columns:
            if row[col] == 1:
                label = col
                break

        if label is None:
            continue

        class_folder = os.path.join(split_path, label)
        os.makedirs(class_folder, exist_ok=True)

        src = os.path.join(split_path, filename)
        dst = os.path.join(class_folder, filename)

        if os.path.exists(src):
            shutil.move(src, dst)

    print(f"{split_path} organized successfully")


base_dir = "Classification-1"

In [22]:
organize_dataset(os.path.join(base_dir, "train"))
organize_dataset(os.path.join(base_dir, "valid"))
organize_dataset(os.path.join(base_dir, "test"))

Classification-1\train organized successfully
Classification-1\valid organized successfully
Classification-1\test organized successfully


In [23]:
for split in ["train", "valid", "test"]:
    path = os.path.join("Classification-1", split)
    classes = [d for d in os.listdir(path)
               if os.path.isdir(os.path.join(path, d))]
    print(split, "classes:", classes)
    print("Number:", len(classes))
    print("-" * 30)

train classes: ['bus', 'car', 'motorcycle', 'truck']
Number: 4
------------------------------
valid classes: ['bus', 'car', 'truck']
Number: 3
------------------------------
test classes: ['bus', 'car', 'truck']
Number: 3
------------------------------


In [24]:
train_dir = r"Classification-1/train"
val_dir   = r"Classification-1/valid"
test_dir  = r"Classification-1/test"

In [25]:
IMG_SIZE = 224
BATCH_SIZE = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

Found 420 files belonging to 4 classes.
Found 40 files belonging to 3 classes.
Found 19 files belonging to 3 classes.


In [26]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y)).prefetch(AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (preprocess_input(x), y)).prefetch(AUTOTUNE)
test_ds = test_ds.map(lambda x, y: (preprocess_input(x), y)).prefetch(AUTOTUNE)


In [1]:
IMG_SIZE = 224

base_model = ResNet50(
    weights='imagenet', 
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = False

ImportError: `load_weights` requires h5py package when loading weights from HDF5. Try installing h5py.

In [ ]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')  
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

In [ ]:
base_model.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)
model.save("resnet_vehicle_model_finetuned.h5")

In [ ]:
model.save("resnet_vehicle_model.h5")